In [7]:
# 把流程濃縮一下（最常見的 join key 清理順序）

# 檢查欄位是否存在（有沒有缺失、是不是該有的 key）
# 檢查唯一性（該唯一的要唯一，不唯一的要理解為什麼）
# 檢查 dtype 一致性（全部轉 str 或 int，避免 join 出錯）
# 檢查 join 對應比例（有多少能對上？有多少對不上？）
# 正式 join，並檢查 join 後的筆數

In [95]:
import pandas as pd
import numpy as np
import ast

ratings = pd.read_csv("row_data/MovieLens 20M Dataset/rating.csv", usecols=["userId", "movieId", "rating", "timestamp"])
movies  = pd.read_csv("row_data/MovieLens 20M Dataset/movie.csv", usecols=["movieId", "title", "genres"])
links   = pd.read_csv("row_data/MovieLens 20M Dataset/link.csv", usecols=["movieId", "imdbId", "tmdbId"])

metadata = pd.read_csv(
    "row_data/The Movies Dataset/movies_metadata.csv",
    low_memory=False,
    usecols=["id", "title", "genres", "overview", "release_date", "runtime", "original_language"]
)
credits  = pd.read_csv("row_data/The Movies Dataset/credits.csv")   # columns: id, title, cast, crew
keywords = pd.read_csv("row_data/The Movies Dataset/keywords.csv")  # columns: id, keywords

# low_memory=False 的作用
# 意思是：不要為了省記憶體而分塊推斷 dtype。
# 改成 一次讀完整個檔案，再統一推斷 dtype。
# 好處：避免 dtype 判斷錯誤、避免很多 DtypeWarning。
# 壞處：可能會用比較多記憶體（因為要一次讀完整檔案來判斷）。

In [96]:
# 1.1. 檢查空值
metadata["id"].isna().sum() # 檢查出來是0

metadata.info() # 但也能這樣直接檢查，看看你要找的欄位跟其他欄位，有沒有Count數字差異

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genres             45466 non-null  object 
 1   id                 45466 non-null  object 
 2   original_language  45455 non-null  object 
 3   overview           44512 non-null  object 
 4   release_date       45379 non-null  object 
 5   runtime            45203 non-null  float64
 6   title              45460 non-null  object 
dtypes: float64(1), object(6)
memory usage: 2.4+ MB


In [105]:
# 1.2.1 檢查重複
metadata['id'].duplicated(keep=False)  # 單純這樣會是一個 mask
metadata['id'].duplicated(keep=False).sum()  # 59 筆重複

dup_ids = metadata[metadata["id"].duplicated(keep=False)].sort_values("id")
dup_ids.info()
# keep=False 這是多個重複都會顯示
# keep="first"（預設）：除了第一次出現以外，其餘重複值標記為 True
# keep="last"：除了最後一次出現以外，其餘重複值標記為 True

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 676 to 16167
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genres             59 non-null     object 
 1   id                 59 non-null     object 
 2   original_language  59 non-null     object 
 3   overview           59 non-null     object 
 4   release_date       59 non-null     object 
 5   runtime            59 non-null     float64
 6   title              59 non-null     object 
dtypes: float64(1), object(6)
memory usage: 3.7+ KB


In [104]:
# 1.2.2 丟掉重複
unique_ids_df = metadata.drop_duplicates(subset=["id"], keep="first")

len(metadata) # 45466
len(unique_ids_df)  # 45436

# unique_ids_df

45436

In [107]:
# 1.3.1 檢查data type，找出奇怪資料 --開始洗資料建立表單

mask = ~unique_ids_df['id'].astype(str).str.isdigit()
rr = unique_ids_df[mask]["id"]

# 下面的更推薦，預計是什麼，其他全部不是的都 coerce
id_numeric = pd.to_numeric(unique_ids_df["id"], errors="coerce").isna()
bad_ids = unique_ids_df.loc[id_numeric, "id"]

bad_ids

19730    1997-08-20
29503    2012-09-29
35587    2014-01-01
Name: id, dtype: object

In [108]:
# 1.3.2 丟掉怪資料，並且轉換data type

subset = unique_ids_df[~unique_ids_df["id"].isin(bad_ids)].copy()
subset["id"] = subset["id"].astype(int)

subset['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45433, dtype: int64

In [138]:
# 1.4 join 前檢查兩者資料
# trans datatype
# links["tmdbId"] = links["tmdbId"].astype('Int64')  ## 很奇怪，這會失敗

links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce").astype('Int64')
subset["id"] = subset["id"].astype('Int64')

# 檢查哪些 tmdbId 存在於 metadata
mask = links["tmdbId"].isin(subset["id"])
mask.mean()  # True=1, False=0, the mean of this will be the actual ratio
# np.float64(0.9866192536109686)

# even detail
# mask.sum()  # check the amount of matching data
# len(mask)   # overall data amount

# links["tmdbId"].size ## 27278

np.float64(0.9866192536109686)

In [134]:
# 5. join，並檢查 join 後的筆數
# 5.1. 確保 dtype 一致
movies["movieId"] = movies["movieId"].astype("Int64")
links["movieId"] = links["movieId"].astype("Int64")
links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce").astype('Int64')
subset["id"] = subset["id"].astype('Int64')

# 5.2. movies + links
movies_links = movies.merge(links[["movieId", "tmdbId"]], on="movieId", how="left")
movies_links

# 5.3. 再跟 metadata join
movies_full = movies_links.merge(
    subset[['genres', 'id', 'original_language', 'overview', 'release_date',
       'runtime', 'title']],
    left_on="tmdbId",
    right_on="id", # we get different col names here, the reason to clearly define left/right
    how="left",    # left join: 保留 MovieLens 所有電影，即使 metadata 沒對上
    suffixes=("_ml", "_tmdb")
)
movies_full

# print("原始 MovieLens 電影數：", movies.shape[0])
# print("join 後電影數：", movies_full.shape[0])
# print(movies_full.head(3))


,movieId,title_ml,genres_ml,tmdbId,genres_tmdb,id,original_language,overview,release_date,runtime,title_tmdb
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,1995-12-22,101.0,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,11862,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,1995-02-10,106.0,Father of the Bride Part II
...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,4436,"[{'id': 35, 'name': 'Comedy'}]",4436,de,The movie deals with a guy who gets to join th...,2007-08-30,90.0,Kein Bund für's Leben
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,9274,"[{'id': 35, 'name': 'Comedy'}]",9274,de,Two friends who are doing civil service flee t...,2002-02-21,83.0,"Fire, Ice & Canned Beer"
27275,131258,The Pirates (2014),Adventure,285213,"[{'id': 28, 'name': 'Action'}, {'id': 36, 'nam...",285213,ko,On the eve of the founding of the Joseon Dynas...,2014-08-06,130.0,The Pirates
27276,131260,Rentun Ruusu (2001),(no genres listed),32099,"[{'id': 10402, 'name': 'Music'}, {'id': 18, 'n...",32099,fi,"The life of Irwin Goodman, a Finnish singer.",2001-01-01,98.0,The Rose of the Rascal


In [ ]:
# mask.mean()  # True=1, False=0, the mean of this will be the actual ratio
# np.float64(0.9866192536109686)

# links["tmdbId"].size ## 27278

# 很奇怪，前面明明沒有完全批配，但是 merge() 的是具又是完全批配的？

# 27278 rows × 11 columns

